# Day7_0: 정적 스크래이핑 (BeautifulSoup)

## 📚 학습 목표

**Part 1: 기초**
1. HTML 구조 이해하기
2. requests로 웹페이지 가져오기
3. BeautifulSoup 파싱 기초
4. find()와 find_all() 사용법 익히기
5. CSS Selector 기본 문법 배우기

**Part 2: 실전 크롤링**
1. CSS Selector 고급 활용하기
2. 테이블 데이터 추출하기
3. 페이지네이션 처리하기
4. 에러 처리 및 재시도 로직 구현하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| HTML 파싱 | 웹사이트 데이터 자동 수집 | 경쟁사 가격 모니터링 |
| CSS Selector | 원하는 요소 정확히 추출 | 뉴스 제목만 추출 |
| 에러 처리 | 안정적인 크롤러 구축 | 네트워크 오류 대응 |
| pandas 연동 | 수집 데이터 즉시 분석 | DataFrame으로 저장 후 시각화 |

**분석가 관점**: API가 없는 웹사이트의 데이터를 자동으로 수집하여 분석할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 HTML 구조 이해

### HTML이란?

**HTML(HyperText Markup Language)**은 웹페이지를 구성하는 마크업 언어입니다.

**기본 구조**:
```html
<태그명 속성="값">내용</태그명>
```

**주요 태그**:
- `<h1>` ~ `<h6>`: 제목 (Heading)
- `<p>`: 문단 (Paragraph)
- `<a>`: 링크 (Anchor)
- `<div>`: 구역 (Division)
- `<table>`: 표
- `<ul>`, `<li>`: 리스트

In [ ]:
# HTML 샘플 데이터
sample_html = """
<html>
    <head>
        <title>쇼핑몰</title>
    </head>
    <body>
        <h1>인기 상품</h1>
        <div class="product">
            <h2>맥북 프로</h2>
            <p class="price">2,500,000원</p>
            <a href="/product/101">상세보기</a>
        </div>
        <div class="product">
            <h2>아이패드</h2>
            <p class="price">900,000원</p>
            <a href="/product/102">상세보기</a>
        </div>
    </body>
</html>
"""

print("HTML 샘플:")
print(sample_html)

### 주요 HTML 속성

| 속성 | 설명 | 예시 |
|------|------|------|
| `class` | CSS 클래스명 | `<div class="product">` |
| `id` | 고유 식별자 | `<div id="main">` |
| `href` | 링크 주소 | `<a href="/page">` |
| `src` | 이미지/스크립트 경로 | `<img src="logo.png">` |

---

## 1.2 requests로 웹페이지 가져오기

### requests 라이브러리

`requests`는 HTTP 요청을 간단하게 보낼 수 있는 Python 라이브러리입니다.

In [ ]:
import requests

# 웹페이지 가져오기
url = "https://example.com"
response = requests.get(url)

# 응답 정보 확인
print(f"상태 코드: {response.status_code}")  # 200: 성공
print(f"인코딩: {response.encoding}")
print(f"내용 길이: {len(response.text)}자")

### HTTP 상태 코드

| 코드 | 의미 | 설명 |
|------|------|------|
| 200 | OK | 요청 성공 |
| 404 | Not Found | 페이지 없음 |
| 403 | Forbidden | 접근 금지 |
| 500 | Server Error | 서버 오류 |

In [ ]:
# HTML 내용 확인
html_content = response.text
print("HTML 앞부분:")
print(html_content[:500])  # 처음 500자만 출력

### 💡 실무 예시: User-Agent 설정

일부 웹사이트는 봇 접근을 막기 위해 User-Agent를 확인합니다.

In [ ]:
# User-Agent 헤더 추가
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
}

response = requests.get(url, headers=headers)
print(f"상태 코드: {response.status_code}")

---

## 1.3 BeautifulSoup 파싱 기초

### BeautifulSoup이란?

HTML/XML을 파싱하여 원하는 데이터를 쉽게 추출할 수 있는 라이브러리입니다.

In [ ]:
from bs4 import BeautifulSoup

# HTML 파싱
soup = BeautifulSoup(sample_html, "html.parser")

# 파싱 결과 확인
print(type(soup))
print(soup.prettify()[:300])  # 정리된 형태로 출력

### 기본 메소드

| 메소드 | 기능 | 반환 |
|--------|------|------|
| `find()` | 첫 번째 요소 찾기 | Tag 객체 또는 None |
| `find_all()` | 모든 요소 찾기 | 리스트 |
| `select()` | CSS Selector로 찾기 | 리스트 |
| `select_one()` | CSS Selector로 첫 요소 | Tag 객체 또는 None |

In [ ]:
# 제목 찾기
title = soup.find("h1")
print(f"제목: {title}")
print(f"제목 텍스트: {title.text}")

In [ ]:
# 모든 h2 태그 찾기
product_names = soup.find_all("h2")
print("상품명 목록:")
for name in product_names:
    print(f"  - {name.text}")

### 💡 실무 예시: 뉴스 헤드라인 추출

In [ ]:
# 뉴스 HTML 샘플
news_html = """
<html>
    <body>
        <div class="news-item">
            <h3 class="title">Python 3.12 출시</h3>
            <p class="summary">새로운 기능 추가</p>
        </div>
        <div class="news-item">
            <h3 class="title">AI 시장 급성장</h3>
            <p class="summary">2024년 전망</p>
        </div>
    </body>
</html>
"""

news_soup = BeautifulSoup(news_html, "html.parser")
headlines = news_soup.find_all("h3", class_="title")

print("뉴스 헤드라인:")
for headline in headlines:
    print(f"  📰 {headline.text}")

---

## 1.4 find()와 find_all() 사용법

### find() - 첫 번째 요소만

조건에 맞는 **첫 번째 요소**를 반환합니다.

In [ ]:
# 첫 번째 상품 div 찾기
first_product = soup.find("div", class_="product")
print(f"첫 번째 상품: {first_product}")

In [ ]:
# 존재하지 않는 태그 찾기
not_found = soup.find("article")
print(f"없는 태그: {not_found}")  # None 반환

### find_all() - 모든 요소

조건에 맞는 **모든 요소**를 리스트로 반환합니다.

In [ ]:
# 모든 상품 div 찾기
all_products = soup.find_all("div", class_="product")
print(f"상품 개수: {len(all_products)}개")

for i, product in enumerate(all_products, 1):
    name = product.find("h2").text
    price = product.find("p", class_="price").text
    print(f"{i}. {name}: {price}")

### 속성으로 찾기

In [ ]:
# class 속성으로 찾기
prices = soup.find_all("p", class_="price")
print("가격 목록:")
for price in prices:
    print(f"  💰 {price.text}")

In [ ]:
# href 속성 추출
links = soup.find_all("a")
print("링크 목록:")
for link in links:
    href = link.get("href")  # 또는 link["href"]
    text = link.text
    print(f"  🔗 {text} → {href}")

### 💡 실무 예시: 제품 정보 구조화

In [ ]:
# 모든 상품 정보를 딕셔너리 리스트로 변환
products = []

for product_div in soup.find_all("div", class_="product"):
    product_info = {
        "name": product_div.find("h2").text,
        "price": product_div.find("p", class_="price").text,
        "link": product_div.find("a").get("href")
    }
    products.append(product_info)

print("수집된 상품 정보:")
for p in products:
    print(p)

---

## 1.5 CSS Selector 기본

### CSS Selector란?

CSS Selector는 HTML 요소를 선택하는 패턴입니다. `select()` 메소드와 함께 사용합니다.

### 기본 선택자

| 선택자 | 설명 | 예시 |
|--------|------|------|
| `태그명` | 태그로 선택 | `div`, `p`, `h1` |
| `.클래스명` | 클래스로 선택 | `.product`, `.price` |
| `#아이디` | ID로 선택 | `#main`, `#header` |
| `부모 > 자식` | 직계 자식 | `div > p` |
| `조상 후손` | 모든 후손 | `div p` |

In [ ]:
# 태그 선택자
all_h2 = soup.select("h2")
print(f"h2 태그 개수: {len(all_h2)}개")

In [ ]:
# 클래스 선택자
price_elements = soup.select(".price")
print("가격 목록 (CSS Selector):")
for price in price_elements:
    print(f"  {price.text}")

In [ ]:
# 자손 선택자
product_links = soup.select("div.product a")
print("상품 링크:")
for link in product_links:
    print(f"  {link.text} → {link.get('href')}")

### select_one() - 첫 번째만

In [ ]:
# 첫 번째 가격만 선택
first_price = soup.select_one(".price")
print(f"첫 번째 가격: {first_price.text}")

### 💡 실무 예시: 복합 선택자

In [ ]:
# product 클래스의 div 안에 있는 h2 태그
product_names = soup.select("div.product h2")

print("상품명 (복합 선택자):")
for name in product_names:
    print(f"  📦 {name.text}")

---

# Part 2: 실전 크롤링

---

## 2.1 CSS Selector 고급 활용

### 속성 선택자

| 선택자 | 설명 | 예시 |
|--------|------|------|
| `[속성]` | 속성이 있는 요소 | `[href]` |
| `[속성=값]` | 속성값이 일치 | `[class="price"]` |
| `[속성^=값]` | 속성값이 시작 | `[href^="http"]` |
| `[속성$=값]` | 속성값이 끝 | `[src$=".jpg"]` |
| `[속성*=값]` | 속성값이 포함 | `[href*="product"]` |

In [ ]:
# href 속성이 있는 모든 요소
all_links = soup.select("[href]")
print(f"링크 개수: {len(all_links)}개")

In [ ]:
# href가 /product로 시작하는 링크
product_links = soup.select('a[href^="/product"]')
print("상품 상세 링크:")
for link in product_links:
    print(f"  {link.get('href')}")

### 가상 선택자

| 선택자 | 설명 | 예시 |
|--------|------|------|
| `:first-child` | 첫 번째 자식 | `div:first-child` |
| `:last-child` | 마지막 자식 | `p:last-child` |
| `:nth-child(n)` | n번째 자식 | `li:nth-child(2)` |

In [ ]:
# 첫 번째 상품 div
first_product = soup.select_one("div.product:first-child")
if first_product:
    print(f"첫 상품: {first_product.find('h2').text}")

### 💡 실무 예시: 복잡한 선택자 조합

In [ ]:
# 복잡한 HTML 샘플
complex_html = """
<div id="products">
    <div class="category" data-id="electronics">
        <h3>전자제품</h3>
        <ul>
            <li class="item" data-price="1000">노트북</li>
            <li class="item" data-price="500">태블릿</li>
        </ul>
    </div>
    <div class="category" data-id="furniture">
        <h3>가구</h3>
        <ul>
            <li class="item" data-price="300">책상</li>
            <li class="item" data-price="200">의자</li>
        </ul>
    </div>
</div>
"""

complex_soup = BeautifulSoup(complex_html, "html.parser")

# data-id가 electronics인 카테고리의 모든 상품
electronics = complex_soup.select('div[data-id="electronics"] li.item')
print("전자제품 목록:")
for item in electronics:
    name = item.text
    price = item.get("data-price")
    print(f"  {name}: {price}만원")

---

## 2.2 테이블 데이터 추출

### HTML 테이블 구조

```html
<table>
  <thead>
    <tr>  <!-- 행(row) -->
      <th>헤더1</th>  <!-- 헤더 셀 -->
      <th>헤더2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>데이터1</td>  <!-- 데이터 셀 -->
      <td>데이터2</td>
    </tr>
  </tbody>
</table>
```

In [ ]:
# 테이블 HTML 샘플
table_html = """
<table id="sales-table">
    <thead>
        <tr>
            <th>지역</th>
            <th>매출</th>
            <th>증감</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>서울</td>
            <td>1,200</td>
            <td class="up">+5%</td>
        </tr>
        <tr>
            <td>부산</td>
            <td>800</td>
            <td class="down">-2%</td>
        </tr>
        <tr>
            <td>대구</td>
            <td>600</td>
            <td class="up">+3%</td>
        </tr>
    </tbody>
</table>
"""

table_soup = BeautifulSoup(table_html, "html.parser")
print("테이블 HTML 파싱 완료")

In [ ]:
# 헤더 추출
headers = [th.text for th in table_soup.select("thead th")]
print(f"헤더: {headers}")

In [ ]:
# 모든 행 추출
rows = []
for tr in table_soup.select("tbody tr"):
    cells = [td.text for td in tr.select("td")]
    rows.append(cells)

print("데이터 행:")
for row in rows:
    print(f"  {row}")

### 💡 실무 예시: pandas DataFrame으로 변환

In [ ]:
import pandas as pd

# DataFrame 생성
df = pd.DataFrame(rows, columns=headers)
print("DataFrame:")
print(df)

# 데이터 타입 확인
print(f"\n데이터 타입:\n{df.dtypes}")

In [ ]:
# 매출 컬럼 정제 (쉼표 제거 후 숫자 변환)
df["매출"] = df["매출"].str.replace(",", "").astype(int)
print("정제된 DataFrame:")
print(df)
print(f"\n총 매출: {df['매출'].sum():,}만원")

---

## 2.3 페이지네이션 처리

### 페이지네이션이란?

여러 페이지에 걸쳐 데이터가 나뉘어 있을 때, 모든 페이지를 순회하며 데이터를 수집하는 기법입니다.

**예시**: `page=1`, `page=2`, `page=3` ...

In [ ]:
# 페이지네이션 URL 패턴 (예시)
base_url = "https://example.com/products"

# 1~3페이지 URL 생성
for page in range(1, 4):
    url = f"{base_url}?page={page}"
    print(f"페이지 {page}: {url}")

### 💡 실무 예시: 여러 페이지 크롤링

In [ ]:
import time

# 여러 페이지 크롤링 시뮬레이션
def crawl_multiple_pages(base_url, max_pages):
    """
    여러 페이지를 순회하며 데이터 수집
    
    Args:
        base_url: 기본 URL
        max_pages: 최대 페이지 수
    
    Returns:
        수집된 데이터 리스트
    """
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"페이지 {page} 크롤링 중...")
        
        # URL 생성
        url = f"{base_url}?page={page}"
        
        # 실제로는 여기서 requests.get(url) 사용
        # response = requests.get(url, headers=headers)
        # soup = BeautifulSoup(response.text, "html.parser")
        
        # 데이터 추출 (시뮬레이션)
        page_data = [f"상품{page}-{i}" for i in range(1, 4)]
        all_data.extend(page_data)
        
        # Rate Limiting (1초 대기)
        time.sleep(1)
    
    return all_data

# 실행 (시뮬레이션)
products = crawl_multiple_pages(base_url, max_pages=3)
print(f"\n총 수집된 상품: {len(products)}개")
print(products)

---

## 2.4 에러 처리 및 재시도 로직

### 왜 에러 처리가 필요한가?

- **네트워크 오류**: 일시적인 연결 실패
- **서버 오류**: 500, 503 등
- **Rate Limiting**: 너무 많은 요청으로 차단
- **요소 없음**: 파싱 중 요소를 찾지 못함

In [ ]:
# 기본 에러 처리
def safe_request(url, max_retries=3):
    """
    안전한 HTTP 요청 (재시도 포함)
    
    Args:
        url: 요청 URL
        max_retries: 최대 재시도 횟수
    
    Returns:
        Response 객체 또는 None
    """
    for attempt in range(1, max_retries + 1):
        try:
            print(f"시도 {attempt}/{max_retries}...")
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # 4xx, 5xx 오류 시 예외 발생
            return response
        
        except requests.exceptions.Timeout:
            print(f"⚠️ 타임아웃 발생 (시도 {attempt})")
        
        except requests.exceptions.HTTPError as e:
            print(f"⚠️ HTTP 오류: {e}")
        
        except requests.exceptions.RequestException as e:
            print(f"⚠️ 요청 오류: {e}")
        
        # 재시도 전 대기 (exponential backoff)
        if attempt < max_retries:
            wait_time = 2 ** attempt  # 2, 4, 8초...
            print(f"  {wait_time}초 후 재시도...")
            time.sleep(wait_time)
    
    print("❌ 모든 재시도 실패")
    return None

# 테스트 (실제로는 유효한 URL 사용)
# response = safe_request("https://example.com")
print("안전한 요청 함수 정의 완료")

### 파싱 에러 처리

In [ ]:
# 안전한 요소 추출
def safe_find_text(soup, selector, default="N/A"):
    """
    안전하게 요소의 텍스트 추출
    
    Args:
        soup: BeautifulSoup 객체
        selector: CSS Selector
        default: 요소가 없을 때 반환값
    
    Returns:
        텍스트 또는 기본값
    """
    element = soup.select_one(selector)
    if element:
        return element.text.strip()
    return default

# 테스트
test_html = "<div><p class='title'>제목</p></div>"
test_soup = BeautifulSoup(test_html, "html.parser")

print(f"존재하는 요소: {safe_find_text(test_soup, 'p.title')}")
print(f"없는 요소: {safe_find_text(test_soup, 'p.subtitle', '제목 없음')}")

### 💡 실무 예시: 견고한 크롤링 함수

In [ ]:
def robust_crawl(url):
    """
    에러 처리가 포함된 견고한 크롤링
    
    Args:
        url: 크롤링할 URL
    
    Returns:
        추출된 데이터 딕셔너리 또는 None
    """
    # 1. 안전한 요청
    response = safe_request(url)
    if not response:
        return None
    
    # 2. 파싱
    try:
        soup = BeautifulSoup(response.text, "html.parser")
    except Exception as e:
        print(f"❌ 파싱 오류: {e}")
        return None
    
    # 3. 안전한 데이터 추출
    data = {
        "title": safe_find_text(soup, "h1"),
        "price": safe_find_text(soup, ".price"),
        "description": safe_find_text(soup, ".description")
    }
    
    return data

# 사용 예시
# result = robust_crawl("https://example.com/product/1")
# if result:
#     print(result)
print("견고한 크롤링 함수 정의 완료")

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. HTML 태그 추출하기 ⭐

**문제**: 다음 HTML에서 `<h1>` 태그의 텍스트를 추출하세요.

```python
html = "<html><body><h1>환영합니다</h1></body></html>"
```

**기대 결과**: `"환영합니다"`

In [ ]:
html = "<html><body><h1>환영합니다</h1></body></html>"
# 여기에 코드를 작성하세요


### Q2. 클래스로 요소 찾기 ⭐⭐

**문제**: 다음 HTML에서 `class="info"` 인 모든 `<p>` 태그의 텍스트를 리스트로 추출하세요.

```python
html = """
<div>
    <p class="info">정보1</p>
    <p>일반 텍스트</p>
    <p class="info">정보2</p>
</div>
"""
```

**기대 결과**: `["정보1", "정보2"]`

In [ ]:
html = """
<div>
    <p class="info">정보1</p>
    <p>일반 텍스트</p>
    <p class="info">정보2</p>
</div>
"""
# 여기에 코드를 작성하세요


### Q3. 링크 속성 추출 ⭐⭐

**문제**: 다음 HTML에서 모든 `<a>` 태그의 `href` 속성을 리스트로 추출하세요.

```python
html = """
<nav>
    <a href="/home">홈</a>
    <a href="/about">소개</a>
    <a href="/contact">연락처</a>
</nav>
"""
```

**기대 결과**: `["/home", "/about", "/contact"]`

In [ ]:
html = """
<nav>
    <a href="/home">홈</a>
    <a href="/about">소개</a>
    <a href="/contact">연락처</a>
</nav>
"""
# 여기에 코드를 작성하세요


### Q4. CSS Selector 사용 ⭐⭐⭐

**문제**: CSS Selector를 사용하여 `class="product"` 인 `<div>` 안의 모든 `<h3>` 태그 텍스트를 추출하세요.

```python
html = """
<div class="product">
    <h3>상품A</h3>
    <p>설명</p>
</div>
<div class="product">
    <h3>상품B</h3>
    <p>설명</p>
</div>
<div class="other">
    <h3>기타</h3>
</div>
"""
```

**기대 결과**: `["상품A", "상품B"]`

In [ ]:
html = """
<div class="product">
    <h3>상품A</h3>
    <p>설명</p>
</div>
<div class="product">
    <h3>상품B</h3>
    <p>설명</p>
</div>
<div class="other">
    <h3>기타</h3>
</div>
"""
# 여기에 코드를 작성하세요


### Q5. 상품 정보 구조화 ⭐⭐⭐

**문제**: 다음 HTML에서 각 상품의 이름과 가격을 딕셔너리 리스트로 추출하세요.

```python
html = """
<div class="item">
    <span class="name">노트북</span>
    <span class="price">1500000</span>
</div>
<div class="item">
    <span class="name">마우스</span>
    <span class="price">30000</span>
</div>
"""
```

**기대 결과**: 
```python
[
    {"name": "노트북", "price": "1500000"},
    {"name": "마우스", "price": "30000"}
]
```

In [ ]:
html = """
<div class="item">
    <span class="name">노트북</span>
    <span class="price">1500000</span>
</div>
<div class="item">
    <span class="name">마우스</span>
    <span class="price">30000</span>
</div>
"""
# 여기에 코드를 작성하세요


### Q6. 속성 선택자 활용 ⭐⭐⭐

**문제**: `href` 속성이 `https`로 시작하는 모든 링크의 텍스트를 추출하세요.

```python
html = """
<a href="https://example.com">외부링크1</a>
<a href="/internal">내부링크</a>
<a href="https://google.com">외부링크2</a>
"""
```

**기대 결과**: `["외부링크1", "외부링크2"]`

In [ ]:
html = """
<a href="https://example.com">외부링크1</a>
<a href="/internal">내부링크</a>
<a href="https://google.com">외부링크2</a>
"""
# 여기에 코드를 작성하세요


### Q7. 테이블 데이터 추출 ⭐⭐⭐⭐

**문제**: 다음 테이블에서 헤더와 데이터를 pandas DataFrame으로 변환하세요.

```python
table_html = """
<table>
    <thead>
        <tr><th>이름</th><th>나이</th><th>직업</th></tr>
    </thead>
    <tbody>
        <tr><td>김철수</td><td>30</td><td>개발자</td></tr>
        <tr><td>이영희</td><td>28</td><td>디자이너</td></tr>
    </tbody>
</table>
"""
```

**기대 결과**: 2행 3열 DataFrame

In [ ]:
table_html = """
<table>
    <thead>
        <tr><th>이름</th><th>나이</th><th>직업</th></tr>
    </thead>
    <tbody>
        <tr><td>김철수</td><td>30</td><td>개발자</td></tr>
        <tr><td>이영희</td><td>28</td><td>디자이너</td></tr>
    </tbody>
</table>
"""
# 여기에 코드를 작성하세요


### Q8. 안전한 요소 추출 ⭐⭐⭐⭐

**문제**: 없을 수도 있는 요소를 안전하게 추출하는 함수를 작성하세요.

- `safe_extract(html, selector)` 함수 구현
- 요소가 있으면 텍스트 반환, 없으면 `"없음"` 반환

```python
html1 = "<div><p class='desc'>설명</p></div>"
html2 = "<div></div>"
```

**기대 결과**: 
- `safe_extract(html1, "p.desc")` → `"설명"`
- `safe_extract(html2, "p.desc")` → `"없음"`

In [ ]:
html1 = "<div><p class='desc'>설명</p></div>"
html2 = "<div></div>"
# 여기에 코드를 작성하세요


### Q9. 페이지네이션 URL 생성 ⭐⭐⭐⭐⭐

**문제**: 1~5페이지의 URL을 생성하고, 각 페이지에서 상품명을 수집하는 시뮬레이션 함수를 작성하세요.

- `base_url = "https://shop.example.com/products"`
- URL 패턴: `{base_url}?page={번호}`
- 각 페이지마다 `f"상품{page}-1"`, `f"상품{page}-2"` 형태로 2개씩 데이터 생성
- 페이지 간 0.5초 대기
- 모든 상품명을 리스트로 반환

**기대 결과**: `["상품1-1", "상품1-2", "상품2-1", "상품2-2", ..., "상품5-2"]` (총 10개)

In [ ]:
base_url = "https://shop.example.com/products"
# 여기에 코드를 작성하세요


### Q10. 종합 크롤링 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 모두 만족하는 크롤링 함수를 작성하세요.

**요구사항**:
1. HTML에서 제품 정보(이름, 가격, 카테고리) 추출
2. 가격에서 쉼표 제거 후 정수 변환
3. pandas DataFrame으로 변환
4. 카테고리별 평균 가격 계산
5. 가장 비싼 제품 정보 출력

```python
html = """
<div class="product" data-category="전자제품">
    <h4>노트북</h4>
    <span class="price">1,500,000</span>
</div>
<div class="product" data-category="전자제품">
    <h4>태블릿</h4>
    <span class="price">800,000</span>
</div>
<div class="product" data-category="가구">
    <h4>책상</h4>
    <span class="price">300,000</span>
</div>
"""
```

**기대 결과**:
- DataFrame 생성 완료
- 카테고리별 평균 가격: 전자제품 1,150,000원, 가구 300,000원
- 최고가 제품: 노트북 (1,500,000원)

In [ ]:
html = """
<div class="product" data-category="전자제품">
    <h4>노트북</h4>
    <span class="price">1,500,000</span>
</div>
<div class="product" data-category="전자제품">
    <h4>태블릿</h4>
    <span class="price">800,000</span>
</div>
<div class="product" data-category="가구">
    <h4>책상</h4>
    <span class="price">300,000</span>
</div>
"""
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| HTML 구조 | `<태그 속성="값">내용</태그>` | 웹페이지 구조 이해 |
| requests | `requests.get(url, headers)` | 웹페이지 가져오기 |
| BeautifulSoup | `BeautifulSoup(html, "html.parser")` | HTML 파싱 |
| find() | `soup.find("태그", class_="클래스")` | 첫 요소 찾기 |
| find_all() | `soup.find_all("태그")` | 모든 요소 찾기 |
| CSS Selector | `soup.select("div.class > p")` | 정교한 요소 선택 |

### Part 2: 실전 크롤링 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 속성 선택자 | `[href^="https"]` | 특정 패턴 URL 필터링 |
| 테이블 추출 | `thead th`, `tbody tr td` | 표 데이터 구조화 |
| pandas 변환 | `pd.DataFrame(rows, columns)` | 수집 데이터 분석 |
| 페이지네이션 | `for page in range(1, n)` | 여러 페이지 순회 |
| 에러 처리 | `try-except`, `response.raise_for_status()` | 안정적인 크롤러 |
| Rate Limiting | `time.sleep(초)` | 서버 부하 방지 |

### 💡 실무 팁

1. **robots.txt 확인**: 크롤링 전 `{사이트}/robots.txt` 확인 필수
2. **User-Agent 설정**: 브라우저처럼 보이도록 헤더 설정
3. **Rate Limiting**: 요청 간 최소 1초 대기 (time.sleep)
4. **에러 처리**: try-except로 안정적인 코드 작성
5. **데이터 검증**: 수집 후 누락/오류 데이터 확인
6. **증분 저장**: 큰 데이터는 주기적으로 파일 저장
7. **법적 검토**: 저작권, 이용약관 준수 확인